<a href="https://colab.research.google.com/github/MasaHirai-Jinen/Medical_Institution_db_colab/blob/main/%E5%8C%BB%E7%99%82%E6%A9%9F%E9%96%A2%E3%82%B3%E3%83%BC%E3%83%89DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

zip_file_path = "jmidb-bucket.zip"
file_id = '1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL' # https://drive.google.com/file/d/1kGQtfh7ZZDHLeK-0_OpFubKod06zb9ZL/view?usp=drive_link
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(zip_file_path)

In [ ]:
import io
import zipfile
import re
from datetime import datetime
import pandas as pd
import sqlite3

### 共有method

In [ ]:
import zipfile
import io
from datetime import datetime
from typing import List, Tuple


def extract_files_from_zip (zip_filepath, filename_in_zip) -> List[Tuple[datetime, str, io.BytesIO]]:
    file_list = []
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for info in zip_ref.infolist():
            l = info.filename.split('/')
            if code_string in l and not info.is_dir():
                i = l.index(code_string)
                category, kikan, get_date, *filename = l[i:]
                get_date = datetime.strptime(get_date, '%Y-%m-%d')
                with zip_ref.open(info) as file:
                    buffer = io.BytesIO(file.read())
                file_list.append((get_date, info.filename, buffer))

    file_list.sort(key=lambda x: x[0])
    return file_list

In [ ]:
import pandas as pd
from typing import NamedTuple

TodofukenType = NamedTuple('TodofukenType', [('id', int), ('name', str)])

class TodofukenDataFrame:
    """
    都道府県データを保持するクラス

    Attributes:
        df (pd.DataFrame): 都道府県データ

    Examples:
        TODOFUKEN = TodofukenDataFrame()
        string = "ｘｘｘｘｘｘｘ kumamoto みえｘｘｘｘｘｘｘｘｘ"
        TODOFUKEN.find_todofuken_from_string(string) # -> TodofukenType(id=24, name='三重県')
    """
    __df: pd.DataFrame = pd.DataFrame(
            [[1,"北海道","北海道", "ほっかいどう", "hokkaido"],
            [2,"青森県","青森","あおもり", "aomori"],
            [3,"岩手県","岩手","いしかわ", "iwate"],
            [4,"宮城県","宮城","みやぎ", "miyagi"],
            [5,"秋田県","秋田","あきた", "akita"],
            [6,"山形県","山形","やまがた", "yamagata"],
            [7,"福島県","福島","ふくい", "fukushima"],
            [8,"茨城県","茨城","いばらき", "ibaraki"],
            [9,"栃木県","栃木","とちぎ", "tochigi"],
            [10,"群馬県","群馬","ぐんま", "gunma"],
            [11,"埼玉県","埼玉","さいたま", "saitama"],
            [12,"千葉県","千葉","ちば", "chiba"],
            [13,"東京都","東京","とうきょう", "tokyo"],
            [14,"神奈川県","神奈川","かながわ", "kanagawa"],
            [15,"新潟県","新潟","にいがた", "niigata"],
            [16,"富山県","富山","とやま", "toyama"],
            [17,"石川県","石川","いしかわ", "ishikawa"],
            [18,"福井県","福井","ふくい", "fukui"],
            [19,"山梨県","山梨","やまなし", "yamanashi"],
            [20,"長野県","長野","ながの", "nagano"],
            [21,"岐阜県","岐阜","ぎふ", "gifu"],
            [22,"静岡県","静岡","しずおか", "shizuoka"],
            [23,"愛知県","愛知","あいち", "aichi"],
            [24,"三重県","三重","みえ", "mie"],
            [25,"滋賀県","滋賀","しが", "shiga"],
            [26,"京都府","京都","きょうと", "kyoto"],
            [27,"大阪府","大阪","おおさか", "osaka"],
            [28,"兵庫県","兵庫","ひょうご", "hyogo"],
            [29,"奈良県","奈良","なら", "nara"],
            [30,"和歌山県","和歌山","わかやま", "wakayama"],
            [31,"鳥取県","鳥取","とっとり", "tottori"],
            [32,"島根県","島根", "しまね", "shimane"],
            [33,"岡山県","岡山","おかやま", "okayama"],
            [34,"広島県","広島","ひろしま", "hiroshima"],
            [35,"山口県","山口","やまぐち", "yamaguchi"],
            [36,"徳島県","徳島","とくしま", "tokushima"],
            [37,"香川県","香川","かがわ", "kagawa"],
            [38,"愛媛県","愛媛","えひめ", "ehime"],
            [39,"高知県","高知","こうち", "kochi"],
            [40,"福岡県","福岡","ふくおか", "fukuoka"],
            [41,"佐賀県","佐賀","さが", "saga"],
            [42,"長崎県","長崎","ながさき", "nagasaki"],
            [43,"熊本県","熊本","くまもと", "kumamoto"],
            [44,"大分県","大分","おおいた", "oita"],
            [45,"宮崎県","宮崎","みやざき", "miyazaki"],
            [46,"鹿児島県","鹿児島","かごしま", "kagoshima"],
            [47,"沖縄県","沖縄","おきなわ", "okinawa"]]
            , columns=["都道府県ID", "都道府県名", "漢字", "ひらがな", "英字"]
        )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_todofuken_from_string(self, string:str) -> TodofukenType|None:
        """
        都道府県名を検索する.
        注意：idが小さいものから順に検索される.

        Args:
            string (str): 検索する文字列

        Returns:
            TodofukenType|None: 検索結果
        """
        string = string.lower()
        mask = self.df[['漢字', 'ひらがな', '英字']].apply(lambda row: any(s in string for s in row), axis=1)
        if mask.any():
            row = self.df[mask].iloc[0]
            return TodofukenType(id = row["都道府県ID"], name = row["都道府県名"])
        return None

TODOFUKEN = TodofukenDataFrame()

In [ ]:
import pandas as pd
from typing import NamedTuple

SyubetuType = NamedTuple('SyubetuType', [('id', int), ('name', str)])

class SyubetuData:
    __df: pd.DataFrame = pd.DataFrame(
        [[1, "医科"],
        [3, "歯科"],
        [4, "薬局"],
        ],
        columns=["種別ID", "種別名"]
    )

    @property
    def df(self) -> pd.DataFrame:
        return self.__df

    def find_syubetu_from_id(self, id:int) -> SyubetuType|None:
        if id in self.__df["種別ID"].values:
            return SyubetuType(id=id, name=self.__df[self.__df["種別ID"] == id]["種別名"].values[0])
        return None

    def find_syubetu_from_name(self, name:str) -> SyubetuType|None:
        if name in self.__df["種別名"].values:
            return SyubetuType(id=self.__df[self.__df["種別名"] == name]["種別ID"].values[0], name=name)
        return None

SYUBETU = SyubetuData()

In [ ]:
from datetime import datetime
import re

def wareki_to_seireki(input_string:str, format_string:str) -> datetime:
    """
    和暦を西暦に変換する

    %g: 年号
    %e: 年
    %m: 月
    %d: 日

    Args:
        input_string  (str): 変換する文字列
        format_string (str): 変換する文字列のフォーマット

    Returns:
        datetime: 変換後の年月日

    Usage Example:
        wareki_to_seireki('昭和元年2月28日', '%g%e年%m月%d日') #-> datetime(1926, 2, 28, 0, 0)
    """
    ERA_DICT = {
        '令和': datetime(2019, 5, 1),
        '平成': datetime(1989, 1, 8),
        '昭和': datetime(1926, 12, 25),
        '大正': datetime(1912, 7, 30),
        '明治': datetime(1868, 1, 25),
    }

    def sort_ear_year(m):
        """
        年号と年をソートする
        return: (年号, 年)
        """
        if m.group(1) in ERA_DICT.keys():
            return m.group(1), m.group(2)
        else:
            return m.group(2), m.group(1)

    # 正規表現のパターンを作成
    pattern = format_string.replace(r'%g', f"({'|'.join(ERA_DICT.keys())})")
    pattern = pattern.replace(r'%e', r'(\d{1,2}|元)')
    pattern = pattern.replace(r'%m', r'\d{1,2}')
    pattern = pattern.replace(r'%d', r'\d{1,2}')

    # 正規表現でマッチ
    match = re.match(pattern, input_string)

    if match is None:
        raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")

    era_name, ear_year = sort_ear_year(match)

    if ear_year == '元':
        ear_year = 0
    else:
        ear_year = int(ear_year)-1
        if ear_year < 0:
            raise ValueError(f"time data '{input_string}' does not match format '{format_string}'")
    year = ERA_DICT[era_name].year + ear_year

    # 新しいformat_stringを作成
    new_format_string = format_string.replace('%g', '').replace('%e', '')
    new_format_string+= "-%Y"

    # マッチした個所を削除したinput_stringを作成
    new_string = match.group(0)

    start , end = match.span(1)
    new_string = new_string[:start] + new_string[end:]
    i = end - start

    start , end = match.span(2)
    new_string = new_string[:start-i] + new_string[end-i:]

    new_string+= f"-{year}"

    return datetime.strptime(new_string, new_format_string)

## DB再作成

### まとめ

**table**
- 医療機関
    - id INTEGER PRIMARY KEY AUTOINCREMENT,
    - コード INTEGER,
    - 名称 TEXT,
    - 住所 TEXT,
    - 郵便番号 TEXT,
    - 電話番号 TEXT,
    - 開設者名 TEXT,
    - 管理者名 TEXT,
    - 指定年月日 TEXT,
    - カテゴリ TEXT,
    - 種別ID INTEGER,
    - 種別名 TEXT,
    - 都道府県ID INTEGER,
    - 都道府県名 TEXT,
    - 作成日 TEXT,
    - ファイル名 TEXT,
    - シート名 TEXT

In [ ]:
from tqdm import tqdm
import sqlite3

db_path = 'jmidb.db'
table_name = '医療機関'

def main():
    table_columns = pd.Index([
        "コード",
        "名称",
        "住所",
        "郵便番号",
        "電話番号",
        "開設者名",
        "管理者名",
        "指定年月日",
        "カテゴリ",
        "種別ID",
        "種別名",
        "都道府県ID",
        "都道府県名",
        "作成日",
        "ファイル名",
        "シート名"
    ])

    create_tabel_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        コード INTEGER,
        名称 TEXT,
        住所 TEXT,
        郵便番号 TEXT,
        電話番号 TEXT,
        開設者名 TEXT,
        管理者名 TEXT,
        指定年月日 TEXT,
        カテゴリ TEXT,
        種別ID INTEGER,
        種別名 TEXT,
        都道府県ID INTEGER,
        都道府県名 TEXT,
        作成日 TEXT,
        ファイル名 TEXT,
        シート名 TEXT
    );
    """

    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
        cursor.execute(create_tabel_query)
        conn.commit()

    return_df = pd.DataFrame()

    file_list = extract_files_from_zip(zip_file_path, 'code')

    for get_date, filename, buffer in tqdm(file_list):
        if re.search(r'歯科併設|医科併設|ikaheisetu|sikaheisetu', filename):
            continue

        dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)

        file_df = pd.DataFrame()

        for sheet_name, df in dfs.items():
            # metaとdataに分割
            i = (df.iloc[:,0]=='1').idxmax()
            meta_df = df.iloc[:i].copy()
            data_df = df.iloc[i:].copy()

            # meta_dfの加工
            meta_df.dropna(axis=0, how='all', inplace=True)
            meta_df.dropna(axis=1, how='all', inplace=True)

            meta_info = meta_df.apply(lambda x: x.str.cat(sep='\t'), axis=1)
            meta_info = meta_info.str.cat(sep='\n')

            # 都道府県の抽出
            todofuken_name = None
            for string in [meta_info, sheet_name, filename]:
                todofuken_name = TODOFUKEN.find_todofuken_from_string(string)
                if isinstance(todofuken_name, TodofukenType):
                    break

            # 種別の抽出
            syubetu = None
            created_date = None

            pattern = r'\[(.*)現在\s(.*)\現存/休止]'
            match_ = re.search(pattern, meta_info)
            if match_:
                created_date, syubetu_name = match_.groups()
                syubetu_name = syubetu_name.strip()
                syubetu = SYUBETU.find_syubetu_from_name(syubetu_name)

            # data_dfの加工
            columns = pd.Index([
                        "項番",
                        "医療機関番号",
                        "医療機関名称",
                        "医療機関所在地",
                        "電話番号／勤務医数",
                        "開設者氏名",
                        "管理者氏名",
                        "指定年月日／登録理由／指定期間始",
                        "病床数／診療科名",
                        "備考"
                        ])

            data_df.columns = columns
            data_df.loc[:,'項番'] = data_df['項番'].ffill()
            data_df.loc[:,'項番'] = data_df['項番'].astype(int)
            groups = data_df.groupby('項番')
            data_df = groups.first()

            sheet_df = data_df.copy()
            sheet_df.loc[:,'種別ID'] = syubetu.id if syubetu else None
            sheet_df.loc[:,'種別名'] = syubetu.name if syubetu else None
            sheet_df.loc[:,'都道府県ID'] = todofuken_name.id if todofuken_name else None
            sheet_df.loc[:,'都道府県名'] = todofuken_name.name if todofuken_name else None
            sheet_df.loc[:,'作成日'] = created_date
            sheet_df.loc[:, 'ファイル名'] = filename
            sheet_df.loc[:, 'シート名'] = sheet_name

            if sheet_df.empty:
                continue
            elif file_df.empty:
                file_df = sheet_df.copy()
            else:
                file_df = pd.concat([file_df, sheet_df])

        if file_df.empty:
            continue

        institution_df = file_df.copy()

        # コード作成
        colname = "医療機関番号"
        if institution_df[[colname, '種別ID', '都道府県ID']].isna().any(axis=1).any():
            raise ValueError("医療機関番号が欠損しています")
        institution_numbers = institution_df[colname].str.replace(r'\D', '', regex=True)
        if (institution_numbers.str.len() == 7).all():
            institution_numbers = institution_numbers.astype(int)
        else:
            raise ValueError("医療機関番号が7桁ではありません")

        code_df = institution_df[['都道府県ID', '種別ID']].copy()
        code_df[colname] = institution_numbers
        code_df = code_df.astype(str)
        code_df['都道府県ID'] = code_df['都道府県ID'].str.zfill(2)
        code_df['種別ID'] = code_df['種別ID'].str.zfill(1)
        code_df[colname] = code_df[colname].str.zfill(7)
        institution_df['コード'] = code_df['都道府県ID'] + code_df['種別ID'] + code_df[colname]
        institution_df['コード'] = institution_df['コード'].astype(int)

        # 郵便番号と住所の分離
        colname = "医療機関所在地"
        address_series = institution_df[colname].copy()

        split_df = address_series.str.extract(r'〒(\d{3}[－-]\d{4})(.*)')
        institution_df[['郵便番号', '住所']] = split_df.copy()
        institution_df['住所'] = institution_df['住所'].str.strip()

        # カテゴリ
        colname = '備考'
        category_series = institution_df[colname].copy()
        category_series = category_series.str.strip()
        category_series = category_series.apply(lambda x: x + '病院' if x in ['特定機能', '地域支援'] else x)
        institution_df['カテゴリ'] = category_series

        # 電話番号の正規化
        colname = '電話番号／勤務医数'
        institution_df["電話番号"] = institution_df[colname].str.replace(r'\D', '-', regex=True)

        # 作成日のdatatimeに変換
        colname = '作成日'
        created_date_series = institution_df[colname].copy()
        created_date_series = created_date_series.str.replace(r'\s', '', regex=True)
        institution_df[colname] = created_date_series.apply(lambda x: wareki_to_seireki(x, '%g%e年%m月%d日'))

        # カラム名変換
        change_columns = {
            "医療機関名称": "名称",
            "開設者氏名": "開設者名",
            "管理者氏名": "管理者名",
            "指定年月日／登録理由／指定期間始": "指定年月日"
        }
        institution_df = institution_df.rename(columns=change_columns)

        # database反映
        with sqlite3.connect(db_path) as conn:
            insert_df = institution_df.copy()

            database_df = pd.read_sql_query(f"SELECT コード, 作成日 FROM {table_name}", conn, dtype=str)
            database_df['コード'] = database_df['コード'].astype(int)
            database_df['作成日'] = pd.to_datetime(database_df['作成日'], format='%Y%m%d')

            merge_df = insert_df.merge(database_df, on=['コード', '作成日'], how='left', indicator=True)
            insert_df = merge_df[merge_df['_merge'] == 'left_only'].drop(columns=['_merge'])

            if insert_df.empty:
                continue

            insert_df['作成日'] = insert_df['作成日'].dt.strftime('%Y%m%d')
            insert_df[table_columns].to_sql(table_name, conn, if_exists='append', index=False)

        if return_df.empty:
            return_df = institution_df.copy()
        else:
            return_df = pd.concat([return_df, institution_df])
    return return_df

return_df = main()
return_df.head()

100%|██████████| 849/849 [49:50<00:00,  3.52s/it]


,医療機関番号,名称,医療機関所在地,電話番号／勤務医数,開設者名,管理者名,指定年月日,病床数／診療科名,備考,種別ID,...,都道府県ID,都道府県名,作成日,ファイル名,シート名,コード,郵便番号,住所,カテゴリ,電話番号
項番,,,,,,,,,,,,,,,,,,,,,
1,01-0041-6,株式会社小林薬局三栄町支店,〒511－0075桑名市三栄町４３,0594-23-5884,株式会社小林薬局 代表取締役 小林 美知子,伊藤 幸作,昭59. 2. 1,None,薬局,4,...,24,三重県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1,2440100416,511－0075,桑名市三栄町４３,薬局,0594-23-5884
2,01-0042-4,クローバー薬局,〒511－0822桑名市馬道１－６９,0594-23-3898,有限会社クローバー薬局 取締役 饗庭 明仁,矢嶋 史郎,昭59. 12. 15,None,薬局,4,...,24,三重県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1,2440100424,511－0822,桑名市馬道１－６９,薬局,0594-23-3898
3,01-0043-2,はやふね薬局,〒511－0808桑名市大字下深谷部７０２,0594-29-1400,早舩 三智也,早舩 三智也,昭60. 3. 1,None,薬局,4,...,24,三重県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1,2440100432,511－0808,桑名市大字下深谷部７０２,薬局,0594-29-1400
4,01-0050-7,有限会社渡辺薬局,〒511－0058桑名市千代田町３７番地,0594-23-1131,有限会社渡辺薬局 取締役 渡邉 和久,渡邉 和久,昭41. 9. 1,None,薬局,4,...,24,三重県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1,2440100507,511－0058,桑名市千代田町３７番地,薬局,0594-23-1131
5,01-0057-2,小林薬局くわな駅西店,〒511－0811桑名市東方１５５－１シャトレ桑名１Ｆ,0594-22-6011,株式会社小林薬局 代表取締役 小林 美知子,三木 邦彦,令元. 5. 1,None,薬局,4,...,24,三重県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1,2440100572,511－0811,桑名市東方１５５－１シャトレ桑名１Ｆ,薬局,0594-22-6011


### 個別確認

In [ ]:
zip_file_path = "/content/jmidb-bucket.zip"
code_string = 'code'
file_list = extract_files_from_zip(zip_file_path, code_string)

file_list[:5]

[(datetime.datetime(2024, 6, 5, 0, 0),
  'content/jmidb-bucket/code/tokaihokuriku/2024-06-05/2406\u3000コード内容別医療機関一覧表（薬局）/2406（三重）コード内容別医療機関一覧表.xlsx',
  <_io.BytesIO at 0x7a9d405519e0>),
 (datetime.datetime(2024, 6, 5, 0, 0),
  'content/jmidb-bucket/code/tokaihokuriku/2024-06-05/2406\u3000コード内容別医療機関一覧表（薬局）/2406（岐阜）コード内容別医療機関一覧表.xlsx',
  <_io.BytesIO at 0x7a9d40551a30>),
 (datetime.datetime(2024, 6, 5, 0, 0),
  'content/jmidb-bucket/code/tokaihokuriku/2024-06-05/2406\u3000コード内容別医療機関一覧表（薬局）/2406（愛知）コード内容別医療機関一覧表.xlsx',
  <_io.BytesIO at 0x7a9d40551a80>),
 (datetime.datetime(2024, 6, 5, 0, 0),
  'content/jmidb-bucket/code/tokaihokuriku/2024-06-05/2406\u3000コード内容別医療機関一覧表（薬局）/2406（富山）コード内容別医療機関一覧表.xlsx',
  <_io.BytesIO at 0x7a9d40551ad0>),
 (datetime.datetime(2024, 6, 5, 0, 0),
  'content/jmidb-bucket/code/tokaihokuriku/2024-06-05/2406\u3000コード内容別医療機関一覧表（薬局）/2406（静岡）コード内容別医療機関一覧表.xlsx',
  <_io.BytesIO at 0x7a9d40551260>)]

In [ ]:
get_date, filename, buffer = file_list[7]
dfs = pd.read_excel(buffer, sheet_name=None, dtype=str)
dfs

{'Sheet1':                     Unnamed: 0         Unnamed: 1            Unnamed: 2  \
 0                          NaN                NaN                   NaN   
 1                コード内容別医療機関一覧表              [岐阜県]                   NaN   
 2                          NaN                NaN                   NaN   
 3     [令和 6年 6月 1日現在　医科　現存/休止]  令和 6年 6月 3日作成　　1頁                   NaN   
 4                           　                 NaN                   NaN   
 ...                        ...                ...                   ...   
 5214                      1402          99,0006,2  独立行政法人国立病院機構長良医療センター   
 5215                       NaN                岐医6                   NaN   
 5216                       NaN                NaN                   NaN   
 5217                       NaN                NaN                   NaN   
 5218                       NaN                NaN                   NaN   
 
                 Unnamed: 3    Unnamed: 4              Unnamed: 5 Unnamed: 6

In [ ]:
sheet_name, df = list(dfs.items())[0]

# metaとdataに分割
i = (df.iloc[:,0]=='1').idxmax()
meta_df = df.iloc[:i].copy()
data_df = df.iloc[i:].copy()

# data_dfの加工
columns = pd.Index([
            "項番",
            "医療機関番号",
            "医療機関名称",
            "医療機関所在地",
            "電話番号／勤務医数",
            "開設者氏名",
            "管理者氏名",
            "指定年月日／登録理由／指定期間始",
            "病床数／診療科名",
            "備考"
            ])

data_df.columns = columns
data_df.loc[:,'項番'] = data_df['項番'].ffill()
data_df.loc[:,'項番'] = data_df['項番'].astype(int)
groups = data_df.groupby('項番')
data_df = groups.first()

# meta_dfの加工
meta_df.dropna(axis=0, how='all', inplace=True)
meta_df.dropna(axis=1, how='all', inplace=True)

meta_info = meta_df.apply(lambda x: x.str.cat(sep='\t'), axis=1)
meta_info = meta_info.str.cat(sep='\n')
print(meta_info)
print('-'*50)
display(data_df)

コード内容別医療機関一覧表	[岐阜県]
[令和 6年 6月 1日現在　医科　現存/休止]	令和 6年 6月 3日作成　　1頁
 　 
--------------------------------------------------


,医療機関番号,医療機関名称,医療機関所在地,電話番号／勤務医数,開設者氏名,管理者氏名,指定年月日／登録理由／指定期間始,病床数／診療科名,備考
項番,,,,,,,,,
1,"01,0011,8",公益社団法人岐阜病院,〒500－8211岐阜市日野東３－１３－６,058(245)8171,公益社団法人岐阜病院 理事長 鈴木 祐一郎,鈴木 祐一郎,昭32. 10. 21,精神 486,病院
2,"01,0219,7",医療法人香風会 黒野病院,〒501－1128岐阜市洞１０２０,058(239)0611,医療法人香風会 理事長 玉木 吉郎,村田 一郎,昭35. 5. 1,精神 250,病院
3,"01,0255,1",岐阜赤十字病院,〒502－0844岐阜市岩倉町３－３６,058(231)2266,日本赤十字社 社長 清家 篤,林 昌俊,昭37. 11. 20,一般 300,地域支援
4,"01,0397,1",近石病院,〒502－0901岐阜市光町２－４６,058(232)2111,医療法人社団登豊会 理事長 近石 登喜雄,清水 克時,昭46. 5. 1,一般 83,病院
5,"01,0416,9",医療法人社団双樹会 早徳病院,〒500－8367岐阜市宇佐南１－８－１,058(272)3253,医療法人社団双樹会 理事長 平野 智久,松原 長樹,昭47. 9. 1,一般 40,病院
...,...,...,...,...,...,...,...,...,...
1398,"34,0033,3",えさきクリニック,〒501－0406本巣市三橋三丁目６０番,058-320-2800,江﨑 孝徳,江﨑 孝徳,平28. 10. 1,脳外 内 外 リハ,診療所
1399,"34,0034,1",糸貫内科クリニック,〒501－0406本巣市三橋２０５番１,058-323-7117,医療法人社団一仁会 理事長 瀬川 孝,瀬川 孝,平29. 3. 1,内 小 皮 美容皮膚科,診療所
1400,"34,0035,8",つじクリニック,〒501－0418本巣市七五三字道下東３１９番地１,058-320-5200,医療法人共心会 理事長 辻 恭嗣,辻 恭嗣,平29. 8. 1,内 外 小 胃腸内科 整外 リハ 皮 肛門外科,診療所


In [ ]:
todofuken_name = None
for string in [meta_info, sheet_name, filename]:
    todofuken_name = TODOFUKEN.find_todofuken_from_string(string)
    if isinstance(todofuken_name, TodofukenType):
        break

print(todofuken_name)

TodofukenType(id=21, name='岐阜県')


In [ ]:
created_date = None
syubetu = None

pattern = r'\[(.*)現在\s(.*)\現存/休止]'
match_ = re.search(pattern, meta_info)
if match_:
    created_date, syubetu_name = match_.groups()
    created_date = created_date.strip()
    syubetu_name = syubetu_name.strip()
    syubetu = SYUBETU.find_syubetu_from_name(syubetu_name)

print(syubetu)

SyubetuType(id=1, name='医科')


In [ ]:
sheet_df = data_df.copy()
sheet_df.loc[:,'種別ID'] = syubetu.id if syubetu else None
sheet_df.loc[:,'種別名'] = syubetu.name if syubetu else None
sheet_df.loc[:,'都道府県ID'] = todofuken_name.id if todofuken_name else None
sheet_df.loc[:,'都道府県名'] = todofuken_name.name if todofuken_name else None
sheet_df.loc[:,'作成日'] = created_date
sheet_df.loc[:, 'ファイル名'] = filename
sheet_df.loc[:, 'シート名'] = sheet_name

sheet_df

,医療機関番号,医療機関名称,医療機関所在地,電話番号／勤務医数,開設者氏名,管理者氏名,指定年月日／登録理由／指定期間始,病床数／診療科名,備考,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
項番,,,,,,,,,,,,,,,,
1,"01,0011,8",公益社団法人岐阜病院,〒500－8211岐阜市日野東３－１３－６,058(245)8171,公益社団法人岐阜病院 理事長 鈴木 祐一郎,鈴木 祐一郎,昭32. 10. 21,精神 486,病院,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
2,"01,0219,7",医療法人香風会 黒野病院,〒501－1128岐阜市洞１０２０,058(239)0611,医療法人香風会 理事長 玉木 吉郎,村田 一郎,昭35. 5. 1,精神 250,病院,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
3,"01,0255,1",岐阜赤十字病院,〒502－0844岐阜市岩倉町３－３６,058(231)2266,日本赤十字社 社長 清家 篤,林 昌俊,昭37. 11. 20,一般 300,地域支援,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
4,"01,0397,1",近石病院,〒502－0901岐阜市光町２－４６,058(232)2111,医療法人社団登豊会 理事長 近石 登喜雄,清水 克時,昭46. 5. 1,一般 83,病院,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
5,"01,0416,9",医療法人社団双樹会 早徳病院,〒500－8367岐阜市宇佐南１－８－１,058(272)3253,医療法人社団双樹会 理事長 平野 智久,松原 長樹,昭47. 9. 1,一般 40,病院,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,"34,0033,3",えさきクリニック,〒501－0406本巣市三橋三丁目６０番,058-320-2800,江﨑 孝徳,江﨑 孝徳,平28. 10. 1,脳外 内 外 リハ,診療所,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1399,"34,0034,1",糸貫内科クリニック,〒501－0406本巣市三橋２０５番１,058-323-7117,医療法人社団一仁会 理事長 瀬川 孝,瀬川 孝,平29. 3. 1,内 小 皮 美容皮膚科,診療所,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1400,"34,0035,8",つじクリニック,〒501－0418本巣市七五三字道下東３１９番地１,058-320-5200,医療法人共心会 理事長 辻 恭嗣,辻 恭嗣,平29. 8. 1,内 外 小 胃腸内科 整外 リハ 皮 肛門外科,診療所,1,医科,21,岐阜県,令和 6年 6月 1日,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1


In [ ]:
institution_df = sheet_df.copy()

# コード作成
colname = "医療機関番号"
institution_numbers = institution_df[colname].str.replace(r'\D', '', regex=True)
if (institution_numbers.str.len() == 7).all():
    institution_numbers = institution_numbers.astype(int)
else:
    raise ValueError("医療機関番号が7桁ではありません")

code_df = institution_df[['都道府県ID', '種別ID']].copy()
code_df[colname] = institution_numbers
code_df = code_df.astype(str)
code_df['都道府県ID'] = code_df['都道府県ID'].str.zfill(2)
code_df['種別ID'] = code_df['種別ID'].str.zfill(1)
code_df[colname] = code_df[colname].str.zfill(7)
institution_df['コード'] = code_df['都道府県ID'] + code_df['種別ID'] + code_df[colname]
institution_df['コード'] = institution_df['コード'].astype(int)

# 郵便番号と住所の分離
colname = "医療機関所在地"
address_series = institution_df[colname].copy()

split_df = address_series.str.extract(r'〒(\d{3}[－-]\d{4})(.*)')
institution_df[['郵便番号', '住所']] = split_df.copy()
institution_df['住所'] = institution_df['住所'].str.strip()

# カテゴリ
colname = '備考'
category_series = institution_df[colname].copy()
category_series = category_series.str.strip()
category_series = category_series.apply(lambda x: x + '病院' if x in ['特定機能', '地域支援'] else x)
institution_df['カテゴリ'] = category_series

# 電話番号の正規化
colname = '電話番号／勤務医数'
institution_df["電話番号"] = institution_df[colname].str.replace(r'\D', '-', regex=True)

# 作成日のdatatimeに変換
colname = '作成日'
created_date_series = institution_df[colname].copy()
created_date_series = created_date_series.str.replace(r'\s', '', regex=True)
institution_df[colname] = created_date_series.apply(lambda x: wareki_to_seireki(x, '%g%e年%m月%d日'))

# カラム名変換
change_columns = {
    "医療機関名称": "名称",
    "開設者氏名": "開設者名",
    "管理者氏名": "管理者名",
    "指定年月日／登録理由／指定期間始": "指定年月日"
}
institution_df = institution_df.rename(columns=change_columns)

table_columns = pd.Index([
    "コード",
    "名称",
    "住所",
    "郵便番号",
    "電話番号",
    "開設者名",
    "管理者名",
    "指定年月日",
    "カテゴリ",
    "種別ID",
    "種別名",
    "都道府県ID",
    "都道府県名",
    "作成日",
    "ファイル名",
    "シート名"
])

institution_df[table_columns]

,コード,名称,住所,郵便番号,電話番号,開設者名,管理者名,指定年月日,カテゴリ,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
項番,,,,,,,,,,,,,,,,
1,2110100118,公益社団法人岐阜病院,岐阜市日野東３－１３－６,500－8211,058-245-8171,公益社団法人岐阜病院 理事長 鈴木 祐一郎,鈴木 祐一郎,昭32. 10. 21,病院,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
2,2110102197,医療法人香風会 黒野病院,岐阜市洞１０２０,501－1128,058-239-0611,医療法人香風会 理事長 玉木 吉郎,村田 一郎,昭35. 5. 1,病院,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
3,2110102551,岐阜赤十字病院,岐阜市岩倉町３－３６,502－0844,058-231-2266,日本赤十字社 社長 清家 篤,林 昌俊,昭37. 11. 20,地域支援病院,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
4,2110103971,近石病院,岐阜市光町２－４６,502－0901,058-232-2111,医療法人社団登豊会 理事長 近石 登喜雄,清水 克時,昭46. 5. 1,病院,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
5,2110104169,医療法人社団双樹会 早徳病院,岐阜市宇佐南１－８－１,500－8367,058-272-3253,医療法人社団双樹会 理事長 平野 智久,松原 長樹,昭47. 9. 1,病院,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1398,2113400333,えさきクリニック,本巣市三橋三丁目６０番,501－0406,058-320-2800,江﨑 孝徳,江﨑 孝徳,平28. 10. 1,診療所,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1399,2113400341,糸貫内科クリニック,本巣市三橋２０５番１,501－0406,058-323-7117,医療法人社団一仁会 理事長 瀬川 孝,瀬川 孝,平29. 3. 1,診療所,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1400,2113400358,つじクリニック,本巣市七五三字道下東３１９番地１,501－0418,058-320-5200,医療法人共心会 理事長 辻 恭嗣,辻 恭嗣,平29. 8. 1,診療所,1,医科,21,岐阜県,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1


In [ ]:
import sqlite3

db_path = 'jmidb.db'

table_columns = pd.Index([
    "コード",
    "名称",
    "住所",
    "郵便番号",
    "電話番号",
    "開設者名",
    "管理者名",
    "指定年月日",
    "カテゴリ",
    "種別ID",
    "種別名",
    "都道府県ID",
    "都道府県名",
    "作成日",
    "ファイル名",
    "シート名"
])

table_name = '医療機関'
create_tabel_query = f"""
CREATE TABLE IF NOT EXISTS {table_name} (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    コード INTEGER,
    名称 TEXT,
    住所 TEXT,
    郵便番号 TEXT,
    電話番号 TEXT,
    開設者名 TEXT,
    管理者名 TEXT,
    指定年月日 TEXT,
    カテゴリ TEXT,
    種別ID INTEGER,
    種別名 TEXT,
    都道府県ID INTEGER,
    都道府県名 TEXT,
    作成日 TEXT,
    ファイル名 TEXT,
    シート名 TEXT
);
"""

with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute(f"DROP TABLE IF EXISTS {table_name}")
    cursor.execute(create_tabel_query)
    conn.commit()

    insert_df = institution_df.copy()

    database_df = pd.read_sql_query(f"SELECT コード, 作成日 FROM {table_name}", conn, dtype=str)
    database_df['コード'] = database_df['コード'].astype(int)
    database_df['作成日'] = pd.to_datetime(database_df['作成日'], format='%Y%m%d')

    merge_df = insert_df.merge(database_df, on=['コード', '作成日'], how='left', indicator=True)
    insert_df = merge_df[merge_df['_merge'] == 'left_only'].drop(columns=['_merge'])

    insert_df['作成日'] = insert_df['作成日'].dt.strftime('%Y%m%d')
    insert_df[table_columns].to_sql(table_name, conn, if_exists='append', index=False)

In [ ]:
with sqlite3.connect(db_path) as conn:
    database_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

database_df

,id,コード,名称,住所,郵便番号,電話番号,開設者名,管理者名,指定年月日,カテゴリ,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
0,1,2110100118,公益社団法人岐阜病院,岐阜市日野東３－１３－６,500－8211,058-245-8171,公益社団法人岐阜病院 理事長 鈴木 祐一郎,鈴木 祐一郎,昭32. 10. 21,病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1,2,2110102197,医療法人香風会 黒野病院,岐阜市洞１０２０,501－1128,058-239-0611,医療法人香風会 理事長 玉木 吉郎,村田 一郎,昭35. 5. 1,病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
2,3,2110102551,岐阜赤十字病院,岐阜市岩倉町３－３６,502－0844,058-231-2266,日本赤十字社 社長 清家 篤,林 昌俊,昭37. 11. 20,地域支援病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
3,4,2110103971,近石病院,岐阜市光町２－４６,502－0901,058-232-2111,医療法人社団登豊会 理事長 近石 登喜雄,清水 克時,昭46. 5. 1,病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
4,5,2110104169,医療法人社団双樹会 早徳病院,岐阜市宇佐南１－８－１,500－8367,058-272-3253,医療法人社団双樹会 理事長 平野 智久,松原 長樹,昭47. 9. 1,病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1398,2113400333,えさきクリニック,本巣市三橋三丁目６０番,501－0406,058-320-2800,江﨑 孝徳,江﨑 孝徳,平28. 10. 1,診療所,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1398,1399,2113400341,糸貫内科クリニック,本巣市三橋２０５番１,501－0406,058-323-7117,医療法人社団一仁会 理事長 瀬川 孝,瀬川 孝,平29. 3. 1,診療所,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1399,1400,2113400358,つじクリニック,本巣市七五三字道下東３１９番地１,501－0418,058-320-5200,医療法人共心会 理事長 辻 恭嗣,辻 恭嗣,平29. 8. 1,診療所,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1400,1401,2119806442,岐阜大学医学部附属病院,岐阜市柳戸１－１,501－1193,058-230-6000,国立大学法人東海国立大学機構 機構長 松尾 清一,秋山 治彦,平16. 5. 20,特定機能病院,1,医科,21,None,20240601,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1


# 抽出

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
from dateutil.relativedelta import relativedelta

db_path = 'test.db'
table_name = '医療機関'

## 全体を抽出

In [ ]:
with sqlite3.connect(db_path) as conn:
    database_df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn, dtype={'コード':int,'種別ID':int,'都道府県ID':int}, parse_dates={'作成日':'%Y%m%d'})

display(database_df.info())
print('-'*100)
print('重複している行数：', database_df.duplicated(subset=['コード', '作成日']).sum())
print('-'*100)
database_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204064 entries, 0 to 1204063
Data columns (total 17 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   id      1204064 non-null  int64         
 1   コード     1204064 non-null  int64         
 2   名称      1204064 non-null  object        
 3   住所      1204064 non-null  object        
 4   郵便番号    1204064 non-null  object        
 5   電話番号    1204014 non-null  object        
 6   開設者名    1204026 non-null  object        
 7   管理者名    1203863 non-null  object        
 8   指定年月日   1204064 non-null  object        
 9   カテゴリ    1204064 non-null  object        
 10  種別ID    1204064 non-null  int64         
 11  種別名     1204064 non-null  object        
 12  都道府県ID  1204064 non-null  int64         
 13  都道府県名   0 non-null        object        
 14  作成日     1204064 non-null  datetime64[ns]
 15  ファイル名   1204064 non-null  object        
 16  シート名    1204064 non-null  object        
dtypes: datet

None

975393


,id,コード,名称,住所,郵便番号,電話番号,開設者名,管理者名,指定年月日,カテゴリ,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
0,1,2440100416,株式会社小林薬局三栄町支店,桑名市三栄町４３,511－0075,0594-23-5884,株式会社小林薬局 代表取締役 小林 美知子,伊藤 幸作,昭59. 2. 1,薬局,4,薬局,24,None,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
1,2,2440100424,クローバー薬局,桑名市馬道１－６９,511－0822,0594-23-3898,有限会社クローバー薬局 取締役 饗庭 明仁,矢嶋 史郎,昭59. 12. 15,薬局,4,薬局,24,None,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
2,3,2440100432,はやふね薬局,桑名市大字下深谷部７０２,511－0808,0594-29-1400,早舩 三智也,早舩 三智也,昭60. 3. 1,薬局,4,薬局,24,None,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
3,4,2440100507,有限会社渡辺薬局,桑名市千代田町３７番地,511－0058,0594-23-1131,有限会社渡辺薬局 取締役 渡邉 和久,渡邉 和久,昭41. 9. 1,薬局,4,薬局,24,None,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1
4,5,2440100572,小林薬局くわな駅西店,桑名市東方１５５－１シャトレ桑名１Ｆ,511－0811,0594-22-6011,株式会社小林薬局 代表取締役 小林 美知子,三木 邦彦,令元. 5. 1,薬局,4,薬局,24,None,2024-06-01,content/jmidb-bucket/code/tokaihokuriku/2024-0...,Sheet1


## 特定月の新規データを抽出

In [ ]:
date_colname = '作成日'
target_year = 2024
target_month = 10

target_date = datetime(target_year, target_month, 1)
target_previous_month = datetime(target_year, target_month, 1) - relativedelta(months=1)

target_query = f"SELECT コード FROM {table_name} WHERE {date_colname} LIKE '{target_date.strftime('%Y%m')}%'"
previous_query = f"SELECT コード FROM {table_name} WHERE {date_colname} LIKE '{target_previous_month.strftime('%Y%m')}%'"

qurey = f"SELECT * FROM {table_name} WHERE コード IN ({target_query} EXCEPT {previous_query}) AND 種別ID = 1"

print(target_date)
print(target_previous_month)
print(target_query)
print(previous_query)
print(qurey)

2024-10-01 00:00:00
2024-09-01 00:00:00
SELECT コード FROM 医療機関 WHERE 作成日 LIKE '202410%'
SELECT コード FROM 医療機関 WHERE 作成日 LIKE '202409%'
SELECT * FROM 医療機関 WHERE コード IN (SELECT コード FROM 医療機関 WHERE 作成日 LIKE '202410%' EXCEPT SELECT コード FROM 医療機関 WHERE 作成日 LIKE '202409%') AND 種別ID = 1


In [ ]:
with sqlite3.connect(db_path) as conn:
    newdata_df = pd.read_sql_query(qurey, conn, dtype={'コード':int,'種別ID':int,'都道府県ID':int}, parse_dates={'作成日':'%Y%m%d'}, index_col='id')

display(newdata_df.info())
newdata_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 374 entries, 987700 to 1201629
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   コード     374 non-null    int64         
 1   名称      374 non-null    object        
 2   住所      374 non-null    object        
 3   郵便番号    374 non-null    object        
 4   電話番号    374 non-null    object        
 5   開設者名    374 non-null    object        
 6   管理者名    374 non-null    object        
 7   指定年月日   374 non-null    object        
 8   カテゴリ    374 non-null    object        
 9   種別ID    374 non-null    int64         
 10  種別名     374 non-null    object        
 11  都道府県ID  374 non-null    int64         
 12  都道府県名   0 non-null      object        
 13  作成日     374 non-null    datetime64[ns]
 14  ファイル名   374 non-null    object        
 15  シート名    374 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(12)
memory usage: 49.7+ KB


None

,コード,名称,住所,郵便番号,電話番号,開設者名,管理者名,指定年月日,カテゴリ,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
id,,,,,,,,,,,,,,,,
987700,1210126250,ありかわ皮フ科・形成外科クリニック千葉駅前,千葉市中央区富士見２－５－１２ ＧＲＡＮＤ ＣＥＮＴＲＡＬ ＣＨＩＢＡ ２０６,260－0015,043-225-0001,有川 俊輔,有川 俊輔,令6. 10. 1,診療所,1,医科,12,None,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
987701,1210126276,そがアイクリニック,千葉市中央区南町２－１５－１９ ＭＴＫビル２階 Ａ室,260－0842,043-312-7011,大岡 恵美,大岡 恵美,令6. 10. 1,診療所,1,医科,12,None,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
988164,1210126284,高洲在宅クリニック,千葉市美浜区高洲３－１４－３ ドレミビル２０１号室,261－0004,043-216-3528,医療法人社団信明会 理事長 橋詰 史朗,橋詰 史朗,令6. 9. 1,診療所,1,医科,12,None,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
988493,1212717718,さんメンタルクリニック,市川市相之川４－１５－２ 吉野ビル３階,272－0143,047-383-9420,宇谷 悦子,宇谷 悦子,令6. 10. 1,診療所,1,医科,12,None,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
988843,1212819282,アリス婦人科ＣＬＩＮＩＣ,船橋市本町６－６－１ 北翔ビル２階,273－0005,047-409-7606,医療法人社団 愛葵会 理事長 弓削田 浩主,山田 大輔,令6. 10. 1,診療所,1,医科,12,None,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1


# 変換

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
from dateutil.relativedelta import relativedelta

db_path = 'jmidb.db'
table_name = '医療機関'

date_colname = '作成日'
target_year = 2024
target_month = 10

target_date = datetime(target_year, target_month, 1)
target_previous_month = datetime(target_year, target_month, 1) - relativedelta(months=1)

target_query = f"SELECT コード FROM {table_name} WHERE {date_colname} LIKE '{target_date.strftime('%Y%m')}%'"
previous_query = f"SELECT コード FROM {table_name} WHERE {date_colname} LIKE '{target_previous_month.strftime('%Y%m')}%'"

qurey = f"SELECT * FROM {table_name} WHERE コード IN ({target_query} EXCEPT {previous_query})"

with sqlite3.connect(db_path) as conn:
    newdata_df = pd.read_sql_query(qurey, conn, dtype={'コード':int,'種別ID':int,'都道府県ID':int}, parse_dates={'作成日':'%Y%m%d'}, index_col='id')

display(newdata_df.info())
newdata_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 903 entries, 902392 to 1127102
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   コード     903 non-null    int64         
 1   名称      903 non-null    object        
 2   住所      903 non-null    object        
 3   郵便番号    903 non-null    object        
 4   電話番号    903 non-null    object        
 5   開設者名    903 non-null    object        
 6   管理者名    903 non-null    object        
 7   指定年月日   903 non-null    object        
 8   カテゴリ    903 non-null    object        
 9   種別ID    903 non-null    int64         
 10  種別名     903 non-null    object        
 11  都道府県ID  903 non-null    int64         
 12  都道府県名   903 non-null    object        
 13  作成日     903 non-null    datetime64[ns]
 14  ファイル名   903 non-null    object        
 15  シート名    903 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(12)
memory usage: 119.9+ KB


None

,コード,名称,住所,郵便番号,電話番号,開設者名,管理者名,指定年月日,カテゴリ,種別ID,種別名,都道府県ID,都道府県名,作成日,ファイル名,シート名
id,,,,,,,,,,,,,,,,
902392,1547141018,浜浦町薬局,新潟市中央区浜浦町１丁目１８１－５,951－8151,025-234-2182,株式会社ケンユウ 代表取締役 原 正,小林 裕代,令6. 9. 1,薬局,4,薬局,15,新潟県,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
902393,1547141026,ミヤサン薬局沼垂店,新潟市中央区沼垂西一丁目８番３号,950－0076,025-243-4570,株式会社ミヤサンファーマシー 代表取締役 宮下 晃一,宮下 晃一,令6. 9. 1,薬局,4,薬局,15,新潟県,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
902422,1547140986,クスリのアオキ船戸山薬局,新潟市江南区船戸山４丁目１０番１６号,950－0153,025-212-9321,株式会社クスリのアオキ 代表取締役 青木 宏憲,三浦 祥宏,令6. 10. 1,薬局,4,薬局,15,新潟県,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
902582,1547140960,真砂薬局,新潟市西区真砂３丁目１１番７号２,950－2074,025-201-7560,株式会社エヌ・エム・アイ 代表取締役 岡本 圭介,坂井 大智,令6. 10. 1,薬局,4,薬局,15,新潟県,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1
902583,1547141034,平島調剤薬局,新潟市西区平島１－２０－７,950－2004,025-233-8972,笹菊薬品株式会社 代表取締役 長澤 吉明,齋藤 大幸,令6. 9. 1,薬局,4,薬局,15,新潟県,2024-10-01,content/jmidb-bucket/code/kantoshinetsu/2024-1...,Sheet1


## PAC

In [ ]:
pac_cloumns = {
    "コード":"InstitutionNo",
    "名称":"InstitutionName",
    "施設名称（略称）":"InstitutionNameShort",
    "施設名称（かな）":"InstitutionKana",
    "郵便番号":"Zip",
    "都道府県ID":"PrefectureId",
    "住所":"Address",
    "電話番号":"Tel",
    "Fax":"Fax",
    "指定年月日":"Url",
    "カテゴリ":"RepresentativeNameSei",
    "作成日":"RepresentativeNameMei"
    }

In [ ]:
pac_df = newdata_df.copy()
# 医科のみ
pac_df = pac_df[pac_df['種別名'] == '医科']

# 必要なカラムを抽出
extract_columns = list(set(pac_cloumns.keys())&set(pac_df.columns))
pac_df = pac_df[extract_columns]

# 足りないカラムを補充
add_columns = list(set(pac_cloumns.keys())-set(pac_df.columns))
pac_df.loc[:, add_columns] = pd.NA

# カラム名を変更
pac_df = pac_df.rename(columns=pac_cloumns)

# ソート
pac_df = pac_df.sort_values(by="InstitutionNo")
pac_df = pac_df[pac_cloumns.values()]

In [ ]:
# zip列の-を削除
pac_df['Zip'] = pac_df['Zip'].str.replace('[-－]', '', regex=True)

# Tel列の-を削除
pac_df['Tel'] = pac_df['Tel'].str.replace('[-－]', '', regex=True)

# RepresentativeNameMei列の日付を%Y%m%dに変換する
pac_df['RepresentativeNameMei'] = pd.to_datetime(pac_df['RepresentativeNameMei'], format='%Y-%m-%d').dt.strftime('%Y%m%d')

pac_df['InstitutionNo'] = pac_df['InstitutionNo'].astype(str).str.zfill(10)

In [ ]:
pac_df.to_csv('pac.csv', index=False)
pac_df

,InstitutionNo,InstitutionName,InstitutionNameShort,InstitutionKana,Zip,PrefectureId,Address,Tel,Fax,Url,RepresentativeNameSei,RepresentativeNameMei
id,,,,,,,,,,,,
1123364,0110419769,ＹＵＵ緩和ケアクリニック,<NA>,<NA>,0060002,1,札幌市手稲区西宮の沢２条４丁目１４番１６号Ａ号室,0116763811,<NA>,令6. 10. 1,診療所,20241001
1123365,0110419777,さっぽろ呼吸器内科クリニック,<NA>,<NA>,0630051,1,札幌市西区宮の沢１条１丁目７番１０号ワイビル宮の沢１階,0116686010,<NA>,令6. 10. 1,診療所,20241001
1123701,0110517018,札幌ひかげ眼科・目もとの美容外科クリニック,<NA>,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３８号,0118371010,<NA>,令6. 10. 1,診療所,20241001
1123703,0110517042,アルバアレルギークリニック札幌,<NA>,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３７号,0115980570,<NA>,令6. 10. 1,診療所,20241001
1123704,0110517059,医療法人 北野通こどもクリニック,<NA>,<NA>,0620053,1,札幌市豊平区月寒東３条１１丁目１－３６ＢＲＡＮＣＨ札幌月寒,0118870422,<NA>,令6. 10. 1,診療所,20241001
...,...,...,...,...,...,...,...,...,...,...,...,...
1105154,4410123659,おおつか内科クリニック,<NA>,<NA>,8700263,44,大分市大字横田７５番地１,0975746020,<NA>,令6. 10. 1,診療所,20241001
1105155,4410123675,ひらかわ産婦人科医院,<NA>,<NA>,8700254,44,大分市横塚２丁目４番５号,0975921000,<NA>,令6. 9. 1,診療所,20241001
1105389,4410412334,日田ごうばる耳鼻科・アレルギー科,<NA>,<NA>,8770072,44,日田市大字十二町３３６番地－１,0973286565,<NA>,令6. 9. 6,診療所,20241001


### 略称



#### [略称変換.xlsx](https://docs.google.com/spreadsheets/d/1sm5L3Q0vsNae7FpkXjPPF0wJm4OLBC7J/edit?usp=drive_link&ouid=117477706610962127029&rtpof=true&sd=true)をダウンロード

In [ ]:
# 略称変換.xlsx をダウンロード
file_id = '1sm5L3Q0vsNae7FpkXjPPF0wJm4OLBC7J'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('略称変換.xlsx')

#### `略称変換.xlsx`の読込

In [ ]:
xlsx_dfs = pd.read_excel('略称変換.xlsx', sheet_name=None)
xlsx_dfs.keys()

dict_keys(['Sheet1', '法人名', '協会名', '略称変換'])

In [ ]:
# 略称変換シートの読込
ryaku_df = xlsx_dfs['略称変換']
ryaku_df['POST'] = ryaku_df['POST'].fillna('').astype(str)
ryaku_df.sort_values(by='PRE', key=lambda x: x.str.len(), ascending=False, inplace=True)

In [ ]:
# 協会名シートから正規表現を作成
kai_df = xlsx_dfs['協会名']

kai_df.dropna(subset="協会名", inplace=True)
kai_df.drop_duplicates(subset="協会名", inplace=True)

kai_df.sort_values(by="協会名", key=lambda x: x.str.len(), ascending=False, inplace=True)

def create_new_column(row):
    """
    協会名と除外列を結合。
    """
    if pd.notna(row['除外']):
        return row['協会名'] + '(?!' + row['除外'] + ')'
    else:
        return row['協会名']

kai_df['pattern'] = kai_df.apply(create_new_column, axis=1)
kai = '|'.join(kai_df['pattern'].to_list())

In [ ]:
# 法人名シートから正規表現を作成
corporation_df = xlsx_dfs['法人名']

corporation_df.sort_values(by="corporation", key=lambda x: x.str.len(), ascending=False, inplace=True)

corporation = "|".join(corporation_df['corporation'].to_list())

#### 略称作成

In [ ]:
df_copy = pac_df.copy()
df_copy['略称'] = df_copy['InstitutionName']

# "法人名 協会名"の削除
setuzokusi = r'[\s・]*'

pattern = re.compile(fr'^((?:{corporation}){setuzokusi}(?:{kai})?{setuzokusi}(?:附属|付属)?{setuzokusi})+')

df_copy['略称'] = df_copy['略称'].str.replace(pattern, '', regex=True)

# JAの変換
df_copy['略称'] = df_copy['略称'].str.replace(r'(^\w*厚生農業協同組合連合会\s*(ＪＡ)?|^ＪＡ\w*厚生連)\s*', 'JA', regex=True)

# 組合の削除
df_copy['略称'] = df_copy['略称'].str.replace(r'^\w*組合立(?<!農業協同組合立)(?<!岡山市久米南町組合立)', '', regex=True)
df_copy['略称'] = df_copy['略称'].str.replace(r'^\w*組合(?:連合会|おおさか)?(?!診療所|総合病院)', '', regex=True)

# 文字列置換
replacement_dict = dict(zip(ryaku_df['PRE'], ryaku_df['POST']))
df_copy['略称'] = df_copy['略称'].replace(replacement_dict, regex=True)


# 空白を削除
df_copy['略称'] = df_copy['略称'].str.strip()

df_copy[['InstitutionName', '略称', 'InstitutionNo']].head()

,InstitutionName,略称,InstitutionNo
id,,,
1123364,ＹＵＵ緩和ケアクリニック,ＹＵＵ緩和ケアクリニック,0110419769
1123365,さっぽろ呼吸器内科クリニック,さっぽろ呼吸器内科クリニック,0110419777
1123701,札幌ひかげ眼科・目もとの美容外科クリニック,札幌ひかげ眼科・目もとの美容外科クリニック,0110517018
1123703,アルバアレルギークリニック札幌,アルバアレルギークリニック札幌,0110517042
1123704,医療法人 北野通こどもクリニック,北野通こどもクリニック,0110517059


In [ ]:
# 略称列をInstitutionNameShortに変換
df_copy['InstitutionNameShort'] = df_copy['略称'].copy()
df_copy.drop(columns=['略称'], inplace=True)
df_copy.head()

,InstitutionNo,InstitutionName,InstitutionNameShort,InstitutionKana,Zip,PrefectureId,Address,Tel,Fax,Url,RepresentativeNameSei,RepresentativeNameMei
id,,,,,,,,,,,,
1123364,0110419769,ＹＵＵ緩和ケアクリニック,ＹＵＵ緩和ケアクリニック,<NA>,0060002,1,札幌市手稲区西宮の沢２条４丁目１４番１６号Ａ号室,0116763811,<NA>,令6. 10. 1,診療所,20241001
1123365,0110419777,さっぽろ呼吸器内科クリニック,さっぽろ呼吸器内科クリニック,<NA>,0630051,1,札幌市西区宮の沢１条１丁目７番１０号ワイビル宮の沢１階,0116686010,<NA>,令6. 10. 1,診療所,20241001
1123701,0110517018,札幌ひかげ眼科・目もとの美容外科クリニック,札幌ひかげ眼科・目もとの美容外科クリニック,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３８号,0118371010,<NA>,令6. 10. 1,診療所,20241001
1123703,0110517042,アルバアレルギークリニック札幌,アルバアレルギークリニック札幌,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３７号,0115980570,<NA>,令6. 10. 1,診療所,20241001
1123704,0110517059,医療法人 北野通こどもクリニック,北野通こどもクリニック,<NA>,0620053,1,札幌市豊平区月寒東３条１１丁目１－３６ＢＲＡＮＣＨ札幌月寒,0118870422,<NA>,令6. 10. 1,診療所,20241001


In [ ]:
df_copy

,InstitutionNo,InstitutionName,InstitutionNameShort,InstitutionKana,Zip,PrefectureId,Address,Tel,Fax,Url,RepresentativeNameSei,RepresentativeNameMei
id,,,,,,,,,,,,
1123364,0110419769,ＹＵＵ緩和ケアクリニック,ＹＵＵ緩和ケアクリニック,<NA>,0060002,1,札幌市手稲区西宮の沢２条４丁目１４番１６号Ａ号室,0116763811,<NA>,令6. 10. 1,診療所,20241001
1123365,0110419777,さっぽろ呼吸器内科クリニック,さっぽろ呼吸器内科クリニック,<NA>,0630051,1,札幌市西区宮の沢１条１丁目７番１０号ワイビル宮の沢１階,0116686010,<NA>,令6. 10. 1,診療所,20241001
1123701,0110517018,札幌ひかげ眼科・目もとの美容外科クリニック,札幌ひかげ眼科・目もとの美容外科クリニック,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３８号,0118371010,<NA>,令6. 10. 1,診療所,20241001
1123703,0110517042,アルバアレルギークリニック札幌,アルバアレルギークリニック札幌,<NA>,0050006,1,札幌市南区澄川６条３丁目２番３７号,0115980570,<NA>,令6. 10. 1,診療所,20241001
1123704,0110517059,医療法人 北野通こどもクリニック,北野通こどもクリニック,<NA>,0620053,1,札幌市豊平区月寒東３条１１丁目１－３６ＢＲＡＮＣＨ札幌月寒,0118870422,<NA>,令6. 10. 1,診療所,20241001
...,...,...,...,...,...,...,...,...,...,...,...,...
1105154,4410123659,おおつか内科クリニック,おおつか内科クリニック,<NA>,8700263,44,大分市大字横田７５番地１,0975746020,<NA>,令6. 10. 1,診療所,20241001
1105155,4410123675,ひらかわ産婦人科医院,ひらかわ産婦人科医院,<NA>,8700254,44,大分市横塚２丁目４番５号,0975921000,<NA>,令6. 9. 1,診療所,20241001
1105389,4410412334,日田ごうばる耳鼻科・アレルギー科,日田ごうばる耳鼻科・アレルギー科,<NA>,8770072,44,日田市大字十二町３３６番地－１,0973286565,<NA>,令6. 9. 6,診療所,20241001


### CSVファイルに出力

In [ ]:
df_copy.to_csv('Q_PAC_utf8.csv', index=False)
df_copy.to_csv('Q_PAC.csv', index=False, encoding='cp932')

## Dowell

In [ ]:
dowell_cloumns = {
    "コード":"医療機関コード",
    "名称":"医療機関名称",
    "郵便番号":"郵便番号",
    "住所":"住所",
    "電話番号":"電話番号",
    "開設者名":"開設者",
    "管理者名":"管理者",
    "指定年月日":"指定年月日",
    "カテゴリ":"備考１",
    "作成日":"備考２"
    }

In [ ]:
dowell_df = newdata_df.copy()

dowell_df = dowell_df[dowell_df['種別名'] == '医科']
# 必要なカラムを抽出
extract_columns = list(set(dowell_cloumns.keys())&set(dowell_df.columns))
dowell_df = dowell_df[extract_columns]

# 足りないカラムを補充
add_columns = list(set(dowell_cloumns.keys())-set(dowell_df.columns))
dowell_df.loc[:, add_columns] = pd.NA

# カラム名を変更
dowell_df = dowell_df.rename(columns=dowell_cloumns)

# ソート
dowell_df = dowell_df.sort_values(by="医療機関コード")
dowell_df = dowell_df[dowell_cloumns.values()]

In [ ]:
dowell_df['備考２'] = pd.to_datetime(dowell_df['備考２'], format='%Y-%m-%d').dt.strftime('%Y%m%d')

# 郵便番号の先頭に"T"を追加
dowell_df['郵便番号'] = '〒' + dowell_df['郵便番号']

# 全てのstr型の 列のエンコーディングエラー対策
str_columns = dowell_df.select_dtypes(include=['object']).columns
# dowell_df[str_columns] = dowell_df[str_columns].map(lambda x:fix_text(x) if x is not None else x)

In [ ]:
dowell_df

,医療機関コード,医療機関名称,郵便番号,住所,電話番号,開設者,管理者,指定年月日,備考１,備考２
id,,,,,,,,,,
1123364,110419769,ＹＵＵ緩和ケアクリニック,〒006－0002,札幌市手稲区西宮の沢２条４丁目１４番１６号Ａ号室,011-676-3811,岡村 直香,岡村 直香,令6. 10. 1,診療所,20241001
1123365,110419777,さっぽろ呼吸器内科クリニック,〒063－0051,札幌市西区宮の沢１条１丁目７番１０号ワイビル宮の沢１階,011-668-6010,医療法人社団Ｍｉｎｄｓ 理事長 多屋 哲也,多屋 哲也,令6. 10. 1,診療所,20241001
1123701,110517018,札幌ひかげ眼科・目もとの美容外科クリニック,〒005－0006,札幌市南区澄川６条３丁目２番３８号,011-837-1010,日景 史人,日景 史人,令6. 10. 1,診療所,20241001
1123703,110517042,アルバアレルギークリニック札幌,〒005－0006,札幌市南区澄川６条３丁目２番３７号,011-598-0570,医療法人社団蓮桜会 理事長 續木 康伸,續木 康伸,令6. 10. 1,診療所,20241001
1123704,110517059,医療法人 北野通こどもクリニック,〒062－0053,札幌市豊平区月寒東３条１１丁目１－３６ＢＲＡＮＣＨ札幌月寒,011-887-0422,医療法人 北野通こどもクリニック 飯田 一樹,飯田 一樹,令6. 10. 1,診療所,20241001
...,...,...,...,...,...,...,...,...,...,...
1105154,4410123659,おおつか内科クリニック,〒870－0263,大分市大字横田７５番地１,097-574-6020,大塚 雄一郎,大塚 雄一郎,令6. 10. 1,診療所,20241001
1105155,4410123675,ひらかわ産婦人科医院,〒870－0254,大分市横塚２丁目４番５号,097-592-1000,医療法人 去定会 理事長 平川 東望子,平川 東望子,令6. 9. 1,診療所,20241001
1105389,4410412334,日田ごうばる耳鼻科・アレルギー科,〒877－0072,日田市大字十二町３３６番地－１,097-328-6565,合原 良亮,合原 良亮,令6. 9. 6,診療所,20241001


In [ ]:
dowell_df.to_csv('Q_Dowell.csv', index=False)